In [1]:
## setup
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext, SparkConf

conf = pyspark.SparkConf().setAppName('Lab1').setMaster('local')
sc = pyspark.SparkContext(conf=conf)

In [2]:
## get the file into hdfs
db = sc.textFile("data/db.txt")

In [3]:
db.take(1)

['A1,B1']

#### 1. List the friends of B1

In [29]:
B1f = db.map(lambda a: (a.split(",")[0],{a.split(",")[1]})).filter(lambda a:a[0]=="B1").reduceByKey(lambda a,b:a.union(b) )
B1f.collect()

[('B1', {'A2', 'B3'})]

#### 2. List the friends of B*

In [30]:
B_f = db.map(lambda a: (a.split(",")[0][0],{a.split(",")[1]})).filter(lambda a:a[0]=="B").reduceByKey(lambda a,b:a.union(b) )
B_f.collect()

[('B', {'A1', 'A2', 'A3', 'B3', 'C3'})]

#### 3. List persons who have C* as their friend.

In [31]:
Casf = db.map(lambda a: (a.split(",")[1][0],{a.split(",")[0]})).filter(lambda a:a[0]=="C").reduceByKey(lambda a,b:a.union(b))
Casf.collect()

[('C', {'A1', 'A3', 'B4'})]

#### 4. List persons who have both C* and A* as their friend.

In [33]:
CAasf = db.map(lambda a: (a.split(",")[1][0],{a.split(",")[0]})).filter(lambda a:a[0]=="C" or a[0]=="A").reduceByKey(lambda a,b:a.union(b)).map(lambda a: ('Frnd to A and C',a[1])).reduceByKey(lambda a,b:a.intersection(b))
CAasf.collect()

[('Frnd to A and C', set())]

#### 5. List persons who have either A* or B* as their friend.

In [5]:
CAasf = db.map(lambda a: (a.split(",")[1][0],{a.split(",")[0]})).filter(lambda a:a[0]=="B" or a[0]=="A").reduceByKey(lambda a,b:a.union(b)).map(lambda a: ('Frnd to A and B',a[1])).reduceByKey(lambda a,b:a.union(b))
CAasf.collect()

[('Frnd to A and B', {'A1', 'A2', 'B1', 'B2', 'B3', 'C1', 'C3', 'C4'})]

#### 6. List all P1,P2 such that P2 is friend of P1 but P1 is not a friend of P2.

In [13]:
One_Sided = db.map(lambda a: (''.join({a.split(",")[0],a.split(",")[1]}),[(a.split(",")[0],a.split(",")[1])] )).reduceByKey(lambda a,b:a+b).filter(lambda a: len(a[1])==1).values()
One_Sided.collect()

[[('A1', 'B1')],
 [('A1', 'C4')],
 [('A2', 'B2')],
 [('A3', 'C1')],
 [('B1', 'A2')],
 [('B1', 'B3')],
 [('B2', 'A1')],
 [('B3', 'A3')],
 [('C1', 'A2')],
 [('C3', 'B2')],
 [('C4', 'A2')],
 [('C3', 'B1')]]

#### 7. List all P1,P2 such that P2 is friend of P1 and P1 is also a friend of P2.

In [16]:
Two_Sided = db.map(lambda a: (''.join({a.split(",")[0],a.split(",")[1]}),[(a.split(",")[0],a.split(",")[1])])).reduceByKey(lambda a,b:a+b).filter(lambda a: len(a[1])==2).map(lambda a: ('Mutual Frnds',a[1][1]))
Two_Sided.collect()

[('Mutual Frnds', ('B3', 'A2')), ('Mutual Frnds', ('C3', 'B4'))]

#### 8. Find the person having maximum number of friends.

In [23]:
max_frnds = db.map(lambda a: (a.split(",")[0], 1)).reduceByKey(lambda a,b:a+b).sortBy(lambda a:a[1], ascending=False)
max_frnds.take(1)

[('C3', 3)]

#### 9. Find the person who is friend to maximum persons.

In [27]:
max_follower = db.map(lambda a: (a.split(",")[1], 1)).reduceByKey(lambda a,b:a+b).sortBy(lambda a:a[1], ascending=False)
max_follower.take(1)

[('A2', 4)]

#### 10. Find popularity rank of each person. Popularity rank(p) = No. of (Friends(Friends(p))

In [57]:
x = db.map(lambda a: (a.split(",")[1], a.split(",")[0]))
y = db.map(lambda a: (a.split(",")[0], {a.split(",")[1]}))
rank_p = x.join(y).values().reduceByKey(lambda a,b:a.union(b)).map(lambda a: (a[0], len(a[1]-{a[0]})))
rank_p.collect()

[('A1', 2),
 ('A2', 2),
 ('B1', 4),
 ('A3', 1),
 ('B3', 2),
 ('C1', 2),
 ('B4', 2),
 ('C3', 3),
 ('C4', 2),
 ('B2', 2)]

#### 11. Find the most popular person based on their popularity rank.

In [61]:
Most_pop = rank_p.sortBy(lambda a:a[1], ascending=False)
Most_pop.take(1)

[('B1', 4)]